### Config

In [8]:
import os
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage

load_dotenv()

GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")

llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    google_api_key=GOOGLE_API_KEY,
    temperature=0.7,
)

In [2]:

# Fungsi tanpa tool
def ask(prompt: str):
    return llm.invoke([HumanMessage(content=prompt)]).content


### Tools

In [3]:
from langchain.agents import initialize_agent, Tool
from langchain.agents.agent_types import AgentType
from langchain_community.tools.tavily_search import TavilySearchResults


# Fungsi dengan tool
def ask_with_search(prompt: str):
    # Setup search tool
    search_tool = TavilySearchResults()
    tools = [
        Tool(
            name="tavily-search",
            func=search_tool.run,
            description="Use this for answering questions about current events or factual information from the web."
        )
    ]

    # Setup agent
    agent = initialize_agent(
        tools,
        llm,
        agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
        verbose=True
    )

    # Jalankan pertanyaan dengan agen
    return agent.run(prompt)

In [4]:
# Pertanyaan biasa (tidak butuh search)
print(ask("Siapa presiden Indonesia sekarang?"))

Presiden Indonesia saat ini adalah Joko Widodo.


In [5]:
# Pertanyaan butuh pencarian web
print(ask_with_search("Siapa presiden Indonesia sekarang?"))



> Entering new AgentExecutor chain...


C:\Users\Fadhil Wicaksono\AppData\Local\Temp\ipykernel_11052\651986331.py:19: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(
C:\Users\Fadhil Wicaksono\AppData\Local\Temp\ipykernel_11052\651986331.py:27: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  return agent.run(prompt)


I need to find out who the current president of Indonesia is. I can use the tavily-search tool to get this information.
Action: tavily-search
Action Input: "who is the current president of indonesia"
Observation: [{'title': 'Indonesia: Prabowo takes office with largest cabinet since 1960s', 'url': 'https://www.bbc.com/news/articles/c756204q2k5o', 'content': "Indonesia leader sworn in with largest cabinet in decades\n\nFormer military general Prabowo Subianto has been sworn in as Indonesia's president, as he announced the country's largest cabinet since the 1960s.\n\nThe 73-year-old, who had been dogged by allegations of human rights abuse for decades, was inagurated on Sunday as the country's eighth president. [...] This spells the end of an era under former leader Joko Widodo, known locally as Jokowi, who presided over a decade of economic growth and infrastructure development.\n\nHaving failed twice to become president, Prabowo finally clawed his way to the highest office after winni

In [6]:
## Tool From Scratch

import os
from datetime import datetime
from dotenv import load_dotenv
from langchain_core.tools import tool
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder



# === TOOL 1: Calculator ===
@tool
def calculator_tool(expression: str) -> str:
    """Evaluasi ekspresi matematika seperti '12 / (2 + 1)'."""
    try:
        result = eval(expression, {"__builtins__": {}})
        return f"Hasil dari '{expression}' adalah {result}"
    except Exception as e:
        return f"Kesalahan menghitung ekspresi: {str(e)}"

# === TOOL 2: Time Now ===
@tool
def time_tool() -> str:
    """Ambil waktu dan tanggal terkini dalam format formal. """
    now = datetime.now()
    return now.strftime("Saat ini: %A, %d %B %Y pukul %H:%M:%S")


In [7]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "Kamu adalah asisten pintar yang bisa menggunakan tools untuk membantu pengguna."),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
])

# === Daftar Tools ===
tools = [calculator_tool, time_tool]

# === Setup Agent ===
agent = create_tool_calling_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

# === Fungsi akhir untuk tanya dengan tools ===
def ask_with_tools(prompt: str):
    return agent_executor.invoke({""
    "input": prompt,
    "chat_history" : []
    })["output"]


In [8]:
print(ask_with_tools("Hitung 123 * (2 + 3)"))



> Entering new AgentExecutor chain...

Invoking: `calculator_tool` with `{'expression': '123 * (2 + 3)'}`


Hasil dari '123 * (2 + 3)' adalah 615123 * (2 + 3) sama dengan 615.

> Finished chain.
123 * (2 + 3) sama dengan 615.


In [9]:
print(ask_with_tools("Tanggal dan jam sekarang berapa?"))



> Entering new AgentExecutor chain...

Invoking: `time_tool` with `{}`


Saat ini: Tuesday, 22 April 2025 pukul 18:18:26Saat ini hari Selasa, 22 April 2025 pukul 18:18:26.

> Finished chain.
Saat ini hari Selasa, 22 April 2025 pukul 18:18:26.


In [10]:
## Tools dokter
doctor_db = [
    {
        "nama": "dr. Andi Wijaya",
        "spesialis": "anak",
        "jadwal": ["Senin", "Rabu", "Jumat"],
        "tersedia": True
    },
    {
        "nama": "dr. Sinta Dewi",
        "spesialis": "gigi",
        "jadwal": ["Selasa", "Kamis"],
        "tersedia": False
    },
    {
        "nama": "dr. Rudi Santoso",
        "spesialis": "mata",
        "jadwal": ["Senin", "Rabu"],
        "tersedia": True
    },
    {
        "nama": "dr. Budi Hartono",
        "spesialis": "umum",
        "jadwal": ["Senin", "Selasa", "Rabu", "Kamis"],
        "tersedia": True
    },
    {
        "nama": "dr. Lisa Marlina",
        "spesialis": "kulit",
        "jadwal": ["Jumat", "Sabtu"],
        "tersedia": False
    }
]


In [11]:
from langchain_core.tools import tool

@tool
def doctor_lookup(spesialis: str, hari: str) -> str:
    """Cek apakah ada dokter dari spesialis tertentu yang tersedia di hari tertentu."""
    spesialis = spesialis.lower()
    hari = hari.capitalize()

    hasil = []
    for dokter in doctor_db:
        if spesialis in dokter["spesialis"].lower() and hari in dokter["jadwal"]:
            status = "tersedia" if dokter["tersedia"] else "tidak tersedia"
            hasil.append(f"{dokter['nama']} ({dokter['spesialis']}) tersedia hari {hari}: {status}")
    
    if not hasil:
        return f"Tidak ditemukan dokter spesialis {spesialis} yang praktek hari {hari}."
    return "\n".join(hasil)


In [12]:
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

tools = [doctor_lookup, time_tool]

prompt = ChatPromptTemplate.from_messages([
    ("system", "1. Jika pengguna tidak menyebutkan hari tertentu, tampilkan seluruh jadwal dokter yang tersedia, lengkap dengan nama dokter dan hari prakteknya. 2. Jika pengguna menyebutkan hari tertentu, tampilkan dokter yang praktek pada hari tersebut. 3. Selalu beri tahu hari ini hari apa sebelum memberikan jadwal dokter 4. Jika dibutuhkan untuk mendapatkan informasi tentang jadwal atau dokter, gunakan tools yang tersedia. 5.Jangan pernah bertanya ulang ke pengguna tentang hari — langsung beri informasi yang relevan."),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
])

agent = create_tool_calling_agent(llm, tools, prompt) #tool calling
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True) #tool calling di log


In [13]:
def ask_dokter(prompt: str):
    return agent_executor.invoke({
        "input": prompt,
        "chat_history": [],
    })["output"]


In [14]:
print(ask_dokter("kapan dokter gigi praktek ?"))



> Entering new AgentExecutor chain...

Invoking: `time_tool` with `{}`


Saat ini: Tuesday, 22 April 2025 pukul 18:18:28
Invoking: `doctor_lookup` with `{'hari': 'Selasa', 'spesialis': 'gigi'}`
responded: Hari ini Selasa, 22 April 2025. Berikut jadwal dokter gigi yang tersedia:

dr. Sinta Dewi (gigi) tersedia hari Selasa: tidak tersedia
Invoking: `doctor_lookup` with `{'hari': 'Rabu', 'spesialis': 'gigi'}`
responded: Hari ini Selasa, 22 April 2025. Berikut jadwal dokter gigi yang tersedia:

Tidak ditemukan dokter spesialis gigi yang praktek hari Rabu.
Invoking: `doctor_lookup` with `{'hari': 'Kamis', 'spesialis': 'gigi'}`
responded: Hari ini Selasa, 22 April 2025. Berikut jadwal dokter gigi yang tersedia:

dr. Sinta Dewi (gigi) tersedia hari Kamis: tidak tersedia
Invoking: `doctor_lookup` with `{'hari': 'Jumat', 'spesialis': 'gigi'}`
responded: Hari ini Selasa, 22 April 2025. Berikut jadwal dokter gigi yang tersedia:

Tidak ditemukan dokter spesialis gigi yang praktek hari Jumat.
Inv

In [15]:
print(ask_dokter("Hari ini dokter umum tersedia enggak ?"))



> Entering new AgentExecutor chain...

Invoking: `time_tool` with `{}`


Saat ini: Tuesday, 22 April 2025 pukul 18:18:31
Invoking: `doctor_lookup` with `{'hari': 'Selasa', 'spesialis': 'Dokter Umum'}`
responded: Hari ini hari Selasa. Mari saya lihat apakah ada dokter umum yang tersedia hari ini.

Tidak ditemukan dokter spesialis dokter umum yang praktek hari Selasa.Hari ini hari Selasa. Tidak ditemukan dokter spesialis dokter umum yang praktek hari ini.

> Finished chain.
Hari ini hari Selasa. Tidak ditemukan dokter spesialis dokter umum yang praktek hari ini.


## Structured Output

In [16]:
from langchain.output_parsers import StructuredOutputParser, ResponseSchema

# 1. Define struktur output (schema)
schemas = [
    ResponseSchema(name="judul", description="Judul buku"),
    ResponseSchema(name="author", description="Penulis buku"),
    ResponseSchema(name="harga", description="Harga buku dalam format mata uang rupiah")
]

In [17]:
# 2. Buat parser
parser = StructuredOutputParser.from_response_schemas(schemas)
format_instructions = parser.get_format_instructions()

In [18]:
# 3. Buat Fungsi
def ask(paragraf: str) -> dict:
    prompt = ChatPromptTemplate.from_messages([
        ("system", "Ekstrak informasi buku dari paragraf berikut."),
        ("user", "{paragraf}\n\n{format_instructions}")
    ])
    
    messages = prompt.format_messages(
        paragraf=paragraf,
        format_instructions=format_instructions
    )
    
    response = llm.invoke(messages)
    return parser.parse(response.content)


In [19]:
paragraf = """
Buku berjudul 'Rahasia Pikiran Positif' karya Dr. Andika Pratama ini sangat menginspirasi. 
Ditawarkan dengan harga terjangkau sekitar Rp89.000, buku ini cocok untuk siapa saja yang ingin membentuk kebiasaan berpikir positif dan hidup lebih bahagia.
"""

In [20]:
hasil = ask(paragraf)
print("Hasil structured output: /n")
print(hasil)


Hasil structured output: /n
{'judul': 'Rahasia Pikiran Positif', 'author': 'Dr. Andika Pratama', 'harga': 'Rp89.000'}


## Embedding Model

In [21]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.schema import Document


# Inisialisasi model embedding dari Google
embedding_model = GoogleGenerativeAIEmbeddings(
    google_api_key=GOOGLE_API_KEY,
    model="models/embedding-001")



In [22]:
data = [
    Document(page_content="Soekarno adalah presiden pertama Indonesia."),
    Document(page_content="Indonesia merdeka pada 17 Agustus 1945."),
    Document(page_content="Resep kue bolu kukus sederhana dengan santan."),
    Document(page_content="Albert Einstein dikenal dengan teori relativitasnya."),
    Document(page_content="ChatGPT adalah model bahasa buatan OpenAI."),
    Document(page_content="Borobudur adalah candi Buddha terbesar di dunia."),
    Document(page_content="Python adalah bahasa pemrograman yang populer."),
    Document(page_content="Rendang termasuk makanan terenak di dunia."),
    Document(page_content="Gempa bumi sering terjadi di wilayah Cincin Api Pasifik."),
    Document(page_content="B.J. Habibie dikenal sebagai bapak teknologi Indonesia."),
]

In [23]:
from langchain.vectorstores import FAISS
from langchain.schema import Document

# Buat vector store dari data dengan embedding model
vectorstore = FAISS.from_documents(data, embedding_model)

# Simpan vectorstore ke lokal
vectorstore.save_local("faiss_index/")



In [ ]:
# Fungsi semantic search
def search_semantic(query: str, k: int = 3):
    results = vectorstore.similarity_search(query, k=k)
    print(f"\n Query: {query}\n")
    for i, result in enumerate(results, 1):
        print(f"{i}. {result.page_content}")

# Fungsi lihat hasil embedding vector
def lihat_embedding(text: str):
    vector = embedding_model.embed_query(text)
    print(f"\n Embedding vector untuk: \"{text}\"\n")
    print(vector[:10], "...")  


In [25]:
# Coba lihat embedding
lihat_embedding("Kapan Indonesia Merdeka")


 Embedding vector untuk: "Kapan Indonesia Merdeka"

[-0.004123105201870203, -0.009060989134013653, -0.030873864889144897, 0.019020926207304, 0.029887637123465538, -0.010326878167688847, -0.02826697565615177, -0.06371747702360153, -0.009733989834785461, 0.03337607532739639] ...


In [26]:
# Coba search
search_semantic("Kapan Indonesia Merdeka")


 Query: Kapan Indonesia Merdeka

1. Indonesia merdeka pada 17 Agustus 1945.
2. Soekarno adalah presiden pertama Indonesia.
3. Resep kue bolu kukus sederhana dengan santan.


In [27]:
# vectorstore = FAISS.load_local("faiss_index/", embedding_model) UP ULANG VECTORSTORE

FAISS (Facebook AI Similarity Search) adalah Library dari Meta AI (Facebook) untuk:
-> Pencarian vektor yang cepat dan efisien (vector similarity search)


Flow : 
Tahap 1 => Embedding Model:

1. Mengubah teks → ke vektor numerik (misal: [0.01, -0.23, 0.45, ...])
2. Vektor ini menangkap makna semantik dari teks (Berdasarkan Similarity)

Tahap 2 => FAISS:

1. Menyimpan semua vektor tadi (dari banyak teks)
2. Saat user memberi query, FAISS akan:
   a. Ubah query → vektor
   b. Cari vektor di database yang paling mirip
   c. Kembalikan hasil dokumen yang relevan


## Agent

In [ ]:
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.tools import tool

@tool
def get_usd_to_idr() -> str:
    """Mengembalikan nilai kurs 1 USD ke IDR saat ini."""
    return "1 USD = 15,200 IDR"

tools = [get_usd_to_idr]

prompt = ChatPromptTemplate.from_messages([
    ("system", "Gunakan tool jika pengguna menanyakan kurs mata uang."),
    ("user", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad")
])

# Buat agent & executor
agent = create_tool_calling_agent(llm=llm, tools=tools, prompt=prompt)
executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

# Fungsi untuk bertanya
def ask_currency_agent(input: str):
    return executor.invoke({"input": input})["output"]

# Test
print(ask_currency_agent("Berapa kurs USD ke IDR hari ini?"))




> Entering new AgentExecutor chain...

Invoking: `get_usd_to_idr` with `{}`


1 USD = 15,200 IDRKurs USD ke IDR hari ini adalah 1 USD = 15.200 IDR.

> Finished chain.
Kurs USD ke IDR hari ini adalah 1 USD = 15.200 IDR.


In [12]:
#Langgraph
""" 
start
  ↓
llm → router → use_tool (jika butuh tool)
             → no_tool (jika tidak)
"""

' \nstart\n  ↓\nllm → router → use_tool (jika butuh tool)\n             → no_tool (jika tidak)\n'

In [16]:
import os
import asyncio
from langchain_core.runnables import RunnableLambda, RunnableBranch, RunnableMap
from langchain_core.messages import HumanMessage, SystemMessage
from langgraph.graph import StateGraph, END

# Tool: fungsi sederhana
def get_usd_to_idr(_: str) -> str:
    return "Saat ini, 1 USD = 15.200 IDR."

# State
class AgentState(dict):
    input: str
    response: str
    use_tool: bool

In [17]:
# LangGraph Node: LLM Reasoning
def llm_node(state: AgentState):
    user_input = state["input"]
    system_prompt = "Jawablah pertanyaan atau jika menyebut 'kurs' atau 'USD', beri sinyal bahwa tool harus digunakan."
    message = [
        SystemMessage(content=system_prompt),
        HumanMessage(content=user_input)
    ]
    result = llm.invoke(message)
    response_text = result.content.lower()

    # Deteksi apakah perlu pakai tool
    state["use_tool"] = any(word in response_text for word in ["tool", "kurs", "usd"])
    state["response"] = result.content
    return state

In [18]:
# LangGraph Node: Tool dipanggil
def use_tool_node(state: AgentState):
    tool_result = get_usd_to_idr(state["input"])
    state["response"] = tool_result
    return state

# LangGraph Node: Jika tool tidak dibutuhkan
def no_tool_node(state: AgentState):
    return state

# Routing: Pilih jalur
def route(state: AgentState):
    return "use_tool" if state["use_tool"] else "no_tool"

In [22]:
# Build Graph
graph = StateGraph(AgentState)

# ambah node
graph.add_node("llm", llm_node)
graph.add_node("tool_handler", use_tool_node)
graph.add_node("no_tool", no_tool_node)

# Set titik masuk awal
graph.set_entry_point("llm")

# Tambahkan percabangan kondisi
graph.add_conditional_edges(
    "llm",
    route,
    {
        "use_tool": "tool_handler",
        "no_tool": "no_tool",
    }
)

# Tentukan akhir dari percabangan
graph.add_edge("tool_handler", END)
graph.add_edge("no_tool", END)

# Compile menjadi agent LangGraph
agent = graph.compile()


In [25]:
# Fungsi akhir untuk dijalankan
async def ask_agent(prompt: str):
    result = await agent.ainvoke({"input": prompt})
    print("Jawaban Agent:", result["response"])

In [28]:
agent.get_graph().print_ascii()


         +-----------+              
         | __start__ |              
         +-----------+              
                *                   
                *                   
                *                   
            +-----+                 
            | llm |                 
            +-----+.                
          ..        ..              
        ..            ..            
       .                .           
+---------+       +--------------+  
| no_tool |       | tool_handler |  
+---------+       +--------------+  
          **        **              
            **    **                
              *  *                  
          +---------+               
          | __end__ |               
          +---------+               


In [26]:
import asyncio
await ask_agent("Berapa kurs USD ke IDR hari ini?")


Jawaban Agent: Saat ini, 1 USD = 15.200 IDR.
